In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("data/clean_data.csv")

In [ ]:
data_geo = data[["Accident_Index","Longitude","Latitude","Local_Authority_(Highway)"]]

In [ ]:
import geopandas
from geodatasets import get_path

In [ ]:
gdf_uk = geopandas.read_file("data/uk_lad.geojson")

In [ ]:
gdf_uk

In [ ]:
gdf_uk_locals = gdf_uk[["reference", "geometry"]]

gdf_uk_locals

In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd

In [ ]:
data = pd.read_csv("data/clean_data.csv")
df = data[["Accident_Index","Longitude","Latitude","Local_Authority_(Highway)"]]

geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

In [ ]:
crs = {'init':'epsg:4326'}
geo_df = gpd.GeoDataFrame(df, #specify our KnC
                          crs=crs, #specify our coordinate reference system
                          geometry=geometry) #specify the geometry list we created
geo_df.head()

#### MOCK

In [ ]:
first = gdf_uk[gdf_uk["reference"] == "E09000020"].index
geo = gdf_uk.iloc[first]["geometry"]
geometry[674].within(geo)

### Show maps with input

In [ ]:
gdf_uk["name"]

In [ ]:
input_district = input()
district = gdf_uk[gdf_uk["name"] == input_district]
code = district["reference"]

In [ ]:
district = data_geo[data_geo["Local_Authority_(Highway)"] == code.values[0]]
district

In [ ]:
district_locs = district[["Latitude","Longitude"]]
district_locs

In [ ]:
import folium
import pandas as pd
import webbrowser
new = 2
# Create a map centered around an initial point
m = folium.Map(location=[district_locs['Latitude'].mean(), district_locs['Longitude'].mean()], zoom_start=12, prefer_canvas=True)

# Add a marker for each data point (you may want to subsample or aggregate your data)
for index, row in district_locs.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,  # Adjust the radius as needed
        fill=True,  # Fill the circle
        color='blue',  # Circle color
        fill_color='blue',  # Fill color
    ).add_to(m)

# Save the map to an HTML file
url = str(code.values[0])+ "_map.html"
webbrowser.open(url,new=new)
m.save(str(code.values[0])+ "_map.html")

### Kensington_and_Chelsea District (E09000020) Accidents in Map

In [ ]:
Kensington_and_Chelsea = data_geo[data_geo["Local_Authority_(Highway)"] == code.values[0]]

In [ ]:
KnC_locs = Kensington_and_Chelsea[["Latitude","Longitude"]]

In [ ]:
# KnC_locs.to_csv(r"data\KnC_locs.csv")

In [ ]:
import folium
import pandas as pd

# Create a map centered around an initial point
m = folium.Map(location=[KnC_locs['Latitude'].mean(), KnC_locs['Longitude'].mean()], zoom_start=15, prefer_canvas=True)

# Add a marker for each data point (you may want to subsample or aggregate your data)
for index, row in KnC_locs.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,  # Adjust the radius as needed
        fill=True,  # Fill the circle
        color='blue',  # Circle color
        fill_color='blue',  # Fill color
    ).add_to(m)

# Save the map to an HTML file
m.save('E09000020_map.html')

### Map Visualizations

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsClassifier
from ipywidgets import interactive
from collections import defaultdict
import hdbscan
import folium
import re
cols = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
        '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', 
        '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
        '#000075', '#808080']*10
sns.set(style="white")

In [ ]:
X = np.array(KnC_locs[["Latitude", "Longitude"]], dtype="float64")
plt.scatter(X[:,0], X[:,1], alpha=0.2, s=50)

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Create a dataframe from your data
KnC = KnC_locs

# Initialize a map centered around the first data point
map_center = [KnC['Latitude'].iloc[0], KnC['Longitude'].iloc[0]]
m = folium.Map(location=map_center, zoom_start=12)

# Create a MarkerCluster layer for the clustered points
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each data point with cluster labels
for _, row in KnC.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], icon=None).add_to(marker_cluster)

# Save the map to an HTML file
m.save('map.html')

### Cluster with DBSCAN

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.cluster import DBSCAN
import random

In [ ]:
X_train = district_locs

In [ ]:
clustering = DBSCAN(eps=0.0006, min_samples=4).fit(X_train)
DBSCAN_dataset = X_train.copy()
DBSCAN_dataset.loc[:,'Cluster'] = clustering.labels_ 

In [ ]:
data2 = DBSCAN_dataset

In [ ]:
# Create a map centered around a specific location (e.g., San Francisco)
m = folium.Map(location=[district_locs['Latitude'].mean(), district_locs['Longitude'].mean()], zoom_start=12, prefer_canvas=True)

# Define colors for clusters
num_clusters = 120
# Generate a list of distinct colors using a colormap
colors = cm.viridis(np.linspace(0, 1, num_clusters))

# Convert the colors to hexadecimal format
cluster_colors = {i + 1: "#{:02x}{:02x}{:02x}".format(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for i, c in enumerate(colors)}

# Add markers to the map with cluster-based colors
for index, row in data2.iterrows():
    cluster = row['Cluster']
    color = cluster_colors.get(cluster, 'gray')
    folium.CircleMarker(location=(row['Latitude'], row['Longitude']),
                        color=color,
                        fill=True,
                        fill_color=color,
                        radius=3).add_to(m)

# Save or display the map
m.save(str(code.values[0])+ "_kmeansclustermap.html")
url = str(code.values[0])+ "_kmeansclustermap.html"
webbrowser.open(url,new=new)

### Cluster with K-Means

In [ ]:
import pandas as pd
import folium
from sklearn.cluster import KMeans
import math 

In [ ]:
# Load your data into a pandas dataframe
data1 = district_locs
# Extract the latitude and longitude columns
X = data1[['Latitude', 'Longitude']]

In [ ]:
# Specify the number of clusters (k)
k = round(math.sqrt(len(X)/2))  # You can adjust this based on your requirements

In [ ]:
# Create a KMeans model
kmeans = KMeans(n_clusters=k, random_state=42)

# Fit the model to your data
kmeans.fit(X)

In [ ]:
# Add the cluster labels to your dataframe
data1['Cluster'] = kmeans.labels_

In [ ]:
m = folium.Map(location=[district_locs['Latitude'].mean(), district_locs['Longitude'].mean()], zoom_start=12, prefer_canvas=True)

# Create points with different random colors for each cluster
cluster_colors = {}
for _, row in data1.iterrows():
    cluster_label = row['Cluster']
    if cluster_label not in cluster_colors:
        cluster_colors[cluster_label] = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    color = cluster_colors[cluster_label]
    folium.Circle(location=[row['Latitude'], row['Longitude']], radius=2, color=color, fill=True, fill_color=color, fill_opacity=0.6, popup=f"Cluster {cluster_label}").add_to(m)

# Save the map to an HTML file
m.save(str(code.values[0])+ "_kmeansclustermap.html")
url = str(code.values[0])+ "_kmeansclustermap.html"
webbrowser.open(url,new=new)